In [40]:
import fitz
import re 

def extract_sentences_from_pdf(pdf_file):
    doc = fitz.open(pdf_file)
    text = ""
    
    for page in doc[5:]:
        text += page.get_text()
        
    #Cleaning the text
    cleaned_text = re.sub(r"[^a-zA-Z\s.]", "", text)
    # Remove extra spaces, including those before periods
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s\.', '.', cleaned_text)
    
    doc.close()
    return cleaned_text

In [41]:
sentences = extract_sentences_from_pdf("./Arjun_Thesis_Proposal.pdf")

In [42]:
print(sentences[:500])

 CHAPTER INTRODUCTION AND BACKGROUND. Biomolecular condensates Biomolecular condensates found in biology commonly function as membraneless organelles that form through liquidliquid phase separation LLPS a process where proteins and nucleic acids as semble into dynamic highlyconcentrated and spatiallysegregated dropletlike structures within cells Figure.. Figure. A cartoon showing a cell with different compartments condensates that organize biomolecules involved in similar functions. The primary 


In [43]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer

In [44]:
tokenizer = Tokenizer()

In [45]:
# Giving number to every unique word in the data. 
tokenizer.fit_on_texts([sentences])

In [50]:
lis = list(tokenizer.word_index.items())[:10]
print(lis)

[('the', 1), ('and', 2), ('of', 3), ('to', 4), ('a', 5), ('in', 6), ('phase', 7), ('for', 8), ('we', 9), ('sequence', 10)]


In [72]:
# Break data into sentences. 
for sentence in sentences.split('.'):
    tokenizer.texts_to_sequences([sentence])

print(tokenizer.texts_to_sequences([sentence])[0])
--> 
[271, 878, 2, 879]
[36, 15, 36, 15, 407, 6, 163, 562, 79, 13, 272, 273, 21, 118, 332, 226, 7, 14, 164, 5, 165, 95, 17, 2, 563, 109, 13, 880, 26, 564, 881, 2, 882, 883, 227, 59, 408, 22]
[]
[22]
[5, 884, 409, 5, 190, 16, 145, 565, 15, 21, 410, 228, 566, 6, 274, 333]
[1, 885, 334, 275, 43, 101, 411, 335, 886, 887, 276, 91, 13, 888, 567, 412, 568, 569, 889, 2, 890, 19, 32, 891, 892, 2, 893, 894, 895, 19, 32, 17, 2, 570, 229, 336, 896, 897, 62, 44, 166, 277]
[27, 19, 278, 4, 1, 8, 898, 3, 571, 572, 45, 573, 22].............